In [1]:
import numpy as np
import os
from pathlib import Path
import tqdm
import matplotlib.pyplot as plt
import random
import tqdm 

%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD

import keras.layers.advanced_activations as advanced_activations
import keras.activations as activations
import keras.layers as layers
from keras.layers import Dense, concatenate
from keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv3D, Conv3DTranspose
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout, Add
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model

Using TensorFlow backend.


In [3]:
tf.__version__

'2.3.0'

### Dataset

In [4]:
past_frames = 10
future_frames = 4
img_size = 256
train_size = 0.9

In [5]:
# handpicked intervals
dataset_intervals = [(300, 800), (300, 800), (390, 900), (300, 800), (390, 900), (390, 850), (350, 800), (350, 900)]

dataset_partitions = []

partition = dict()
labels = dict()

for di in dataset_intervals:

    # sizes
    size = di[1] - di[0] - past_frames - future_frames
    train_len = int(size * train_size)
    
    partition_raw = []
    
    # per ogni dataset genera la lista di ids
    for i in range(size):
        #partition["id-{}".format(i)] = list(range(i, i + past_frames))
        partition_raw.append("id-{}".format(i))
        labels["id-{}".format(i)] = list(range(i + past_frames, i + past_frames + future_frames))
        
    partition["train"] = partition_raw[:train_len]
    partition["validation"] = partition_raw[train_len:] 
    
    # lista di (partition, labels)
    dataset_partitions.append((partition, labels))

In [6]:
import numpy as np

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, filename, list_IDs, labels, batch_size=16, input_dim=(past_frames, img_size, img_size),  output_dim=(future_frames, img_size, img_size), n_channels=1, shuffle=True):
        'Initialization'

        self.filename = filename

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.batch_size = batch_size

        self.list_IDs = list_IDs
        self.labels = labels

        self.n_channels = n_channels
        self.shuffle = shuffle

        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        
        # indice di un batch

        # Generate indexes of the batch
        # indexes = shuffle(list_IDs)
        indexes = self.indexes[ index * self.batch_size : (index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X = np.empty((self.batch_size, *self.input_dim, self.n_channels))
        y = np.empty((self.batch_size, *self.output_dim, self.n_channels))

        src = np.load(self.filename)

        # Generate data for i IDs
        # id: 0 +
        for i, ID in enumerate(list_IDs_temp):

            # partition[id] -> start position for train window
            r = src[i: i + past_frames]
            X[i,] = r.reshape(r.shape[0], r.shape[1], r.shape[2], 1)

            # labels[id] -> list of frame indices for predict window
            r = src[self.labels[ID]]
            y[i] = r.reshape(r.shape[0], r.shape[1], r.shape[2], 1)

        return X, y

In [7]:
ds_1_train_generator = DataGenerator("../datasets/baganza/mini-407-662-397-652.npy", dataset_partitions[0][0]['train'], dataset_partitions[0][1])
ds_1_test_generator = DataGenerator("../datasets/baganza/mini-407-662-397-652.npy", dataset_partitions[0][0]['validation'], dataset_partitions[0][1])

In [8]:
ds_1_train_generator.__getitem__(10)[0].shape

(16, 10, 256, 256, 1)

In [9]:
type(ds_1_test_generator.__getitem__(10)[1])

numpy.ndarray

### Model

In [10]:
sample_shape = (10, 256, 256, 1)

# Create the model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(AveragePooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(AveragePooling3D(pool_size=(2, 2, 2)))

model.add(Conv3DTranspose(filters=64, kernel_size=(3,3,3), strides=(2,2,2), padding="same",activation='relu'))
model.add(Conv3DTranspose(filters=32, kernel_size=(3,3,3), strides=(1,2,2), padding="same",activation='relu'))
model.add(Conv3DTranspose(filters=1, kernel_size=(3,3,3), strides=(1,1,1), padding="same",activation='relu'))

model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 10, 256, 256, 32)  896       
_________________________________________________________________
average_pooling3d (AveragePo (None, 5, 128, 128, 32)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 128, 128, 64)   55360     
_________________________________________________________________
average_pooling3d_1 (Average (None, 2, 64, 64, 64)     0         
_________________________________________________________________
conv3d_transpose (Conv3DTran (None, 4, 128, 128, 64)   110656    
_________________________________________________________________
conv3d_transpose_1 (Conv3DTr (None, 4, 256, 256, 32)   55328     
_________________________________________________________________
conv3d_transpose_2 (Conv3DTr (None, 4, 256, 256, 1)    8

In [11]:
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = "mean_squared_error", metrics = ["accuracy"])

In [14]:
tf.get_logger().setLevel('INFO')

In [15]:
model.fit(x = ds_1_train_generator,
                    validation_data = ds_1_test_generator,
                    use_multiprocessing = True,
                    workers = 6,
         epochs = 50)

Epoch 1/50
30/30 [==============================] - 25s 819ms/step - loss: 0.1600 - accuracy: 0.8135 - val_loss: 0.6136 - val_accuracy: 0.5363
Epoch 2/50
30/30 [==============================] - 24s 814ms/step - loss: 0.1600 - accuracy: 0.8134 - val_loss: 0.5961 - val_accuracy: 0.5338
Epoch 3/50
30/30 [==============================] - 25s 821ms/step - loss: 0.1598 - accuracy: 0.8132 - val_loss: 0.5890 - val_accuracy: 0.5330
Epoch 4/50
30/30 [==============================] - 24s 809ms/step - loss: 0.1585 - accuracy: 0.8135 - val_loss: 0.5879 - val_accuracy: 0.5332
Epoch 5/50
30/30 [==============================] - 24s 810ms/step - loss: 0.1609 - accuracy: 0.8133 - val_loss: 0.6089 - val_accuracy: 0.5343
Epoch 6/50
 4/30 [===>..........................] - ETA: 16s - loss: 0.1493 - accuracy: 0.8200

Process Keras_worker_ForkPoolWorker-134:
Process Keras_worker_ForkPoolWorker-135:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-137:
  File "/home/diego/anaconda3/envs/parflood/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/diego/anaconda3/envs/parflood/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Keras_worker_ForkPoolWorker-138:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/diego/anaconda3/envs/parflood/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/diego/anaconda3/envs/parflood/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-136:
  File "/home/diego/anaconda3/envs/parflood/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (mos

KeyboardInterrupt: 